In [218]:
# !pip install slack_sdk

In [219]:
import os
from slack_sdk import WebClient

token = os.getenv("SLACK_TOKEN")
d_cookie = os.getenv("SLACK_D_COOKIE")

In [220]:
headers = {
    'Cookie': f'd={d_cookie}',
    'User-Agent': 'Mozilla/5.0 (compatible; Python)'
}   
client = WebClient(token=token,headers=headers)




In [ ]:
response = client.chat_postMessage(
    channel="D7GH378QG",
    text="test message from python slack bot"
)
print(response)


In [ ]:
response = client.conversations_list(types="im")


In [ ]:
response = client.conversations_history(channel="D01LM1NMJ0J", limit=3)
print(response["messages"])


In [ ]:
response = client.conversations_list(types="im,private_channel,public_channel,mpim")


In [ ]:
response = client.conversations_list(types="im")


In [ ]:
users = client.users_list()

# Compute mapping

In [222]:
def compute_channelid_to_name(client):
    channel_to_name = {}

    # Step 1: Get all users (one request)
    users_response = client.users_list()
    user_id_to_real_name = {
        user["id"]: user["real_name"]
        for user in users_response["members"]
        if not user.get("deleted", False)
    }

    channels = []
    cursor = None
    while True:
        response = client.conversations_list(types="im,mpim,public_channel,private_channel", limit=1000, cursor=cursor)
        channels.extend(response["channels"])
        cursor = response.get("response_metadata", {}).get("next_cursor")
        if not cursor:
            break
    

    for channel in channels:
        if channel.get("is_im", False):
            channel_id = channel["id"]
            user_id = channel["user"]
            real_name = user_id_to_real_name.get(user_id, "Unknown User")
            channel_to_name[channel_id] = real_name
        elif channel.get("is_mpim", False):
            channel_id = channel["id"]
            channel_name = channel["name"]
            channel_to_name[channel_id] = channel_name
        else:
            #public or private channel
            channel_id = channel["id"]
            channel_name = channel["name"]
            channel_to_name[channel_id] = channel_name

    return channel_to_name, user_id_to_real_name

In [223]:
channel_id_to_name, user_id_to_real_name = compute_channelid_to_name(client)

In [293]:
def get_starred_channel_ids(client):
    # Get starred channels for the authenticated user
    stars_response = client.stars_list(limit=1000)
    starred_channels = [
        item["channel"]
        for item in stars_response.get("items", [])
        # if item.get("type") == "channel"
    ]
    return starred_channels

In [288]:
channel = "C095U8548RW"

In [289]:
channel_id_to_name[channel]

'2026-team-offsite'

In [290]:
float(client.conversations_history(channel="C095U8548RW", limit=10)["messages"][1]["ts"]) > last_week_timestamp

True

In [311]:
def list_messages(channel_id):
    res = client.conversations_history(channel=channel_id, limit=5)
    print(channel_id_to_name[channel_id])
    for msg in res["messages"]:
        user_id = msg.get('user', 'unknown user')
        user_name = user_id_to_real_name.get(user_id, "Unknown User")
        print(f"@{user_name}: {msg['text']}")

list_messages(starred_channels[15])

team-engineering-leads
@Shubham Gupta: <@U8KUAD396> has left the channel
@Madhava Jay: <@UQCEALEPP> has left the channel
@Thiago Costa Porto: <@UJULKVAAV> has left the channel
@Madhava Jay: Looks like curry! :heart:
@Thiago Costa Porto: <@U01SAESBJA0> <@U8KUAD396> 


In [295]:
len(starred_channels)

41

In [294]:
starred_channels = get_starred_channel_ids(client)

/Users/koen/miniconda3/envs/syftbox-mcp/lib/python3.12/site-packages/slack_sdk/web/deprecation.py:44: UserWarning: stars.list is deprecated. For more info, go to https://api.slack.com/changelog/2023-07-its-later-already-for-stars-and-reminders
  warnings.warn(message)


In [257]:
assert len(starred_channels) == len(set(starred_channels).intersection(channel_id_to_name.keys()))

In [258]:
import time
last_week_timestamp = int(time.time()) - 60 * 60 * 24 * 7

In [312]:
from collections import defaultdict

In [318]:
unread_messages = defaultdict(list)
for channel in starred_channels:
    
    res = client.conversations_history(channel=channel, limit=100)
    for msg in res["messages"]:
        if float(msg["ts"]) > last_week_timestamp:
            if msg.get("subtype") == "channel_join":
                continue
            
            user_id = msg.get("user", None)
            sender_name = user_id_to_real_name.get(user_id, "Unknown User")
            channel_name = channel_id_to_name.get(channel, None)
            if channel_name is None:
                channel_name = "unknown channel"
            
            
            unread_messages[channel_name].append({"user": sender_name, "text": msg["text"]})

    # print(unread_messages)
    

In [319]:
for channel_name, msgs in unread_messages.items():
    print(f"in #{channel_name}")
    for msg in reversed(msgs):    
        print(f"@{msg['user']}: {msg['text']}")
    print("="*100)

in #2026-team-offsite
@Slackbot: 
@Irina Bejan: :party_parrot:
@Irina Bejan: :palm_tree:
@Unknown User: Hi, everyone! Please welcome <@U0191FVFKRP>! Here’s a little bit about them, in their own words:

• *Role:* Contributor
• *Pronouns:* organiser/organis'em
• *Current project:* 
Make our offsite great!
• *Location:* 
Switzerland
• *Bio:* 
I love offsites :saluting_face:
• *Fun fact:* 
Never been on an offsite with the OM team :see_no_evil:
@Dave Buckley: Hyped for this
@Andrew Trask: the party planning committee
@Unknown User: Hi, everyone! Please welcome <@U07JZQYGTNC>! Here’s a little bit about them, in their own words:

• *Role:* executive stalker
• *Pronouns:* she/her
• *Current project:* 
corporate party
• *Location:* 
:flag-br:
• *Bio:* 
I like rainbows and butterflies and serial killers
• *Fun fact:* 
I have uneven thumbs :+1::skin-tone-3:
@Slackbot: 
@Lacey Strahm: Hi all - thanks for raising your hand and expressing your interest in planning a team off-site for 2026. This is 

In [102]:
res["messages"]

[{'user': 'U7GH375CG',
  'type': 'message',
  'ts': '1752478898.253609',
  'client_msg_id': '5bf65986-3ca0-4c43-aa0e-42512ffe5e98',
  'text': 'perhaps something like this could be a use case for syftbox\n<https://x.com/kevinnguyendn/status/1941497299320107172>',
  'team': 'T6963A864',
  'thread_ts': '1752478898.253609',
  'reply_count': 2,
  'reply_users_count': 1,
  'latest_reply': '1752479045.169039',
  'reply_users': ['U7GH375CG'],
  'is_locked': False,
  'subscribed': True,
  'last_read': '1752479045.169039',
  'attachments': [{'image_url': 'https://pbs.twimg.com/media/GvE8ykFa4AAOITJ.png:large',
    'image_width': 600,
    'image_height': 332,
    'image_bytes': 14659,
    'from_url': 'https://x.com/kevinnguyendn/status/1941497299320107172',
    'service_icon': 'http://abs.twimg.com/favicons/twitter.3.ico',
    'id': 1,
    'original_url': 'https://x.com/kevinnguyendn/status/1941497299320107172',
    'fallback': 'X (formerly Twitter): andy nguyen (@kevinnguyendn) on X',
    'text'

## Appendix

In [85]:
# channel_id_to_user_real_name

In [ ]:
# Format the date 7 days ago in YYYY-MM-DD
seven_days_ago_date = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")

# Search query: messages mentioning your user ID, after the date
query = f"@{my_user_ida}"


# Call search.messages API
result = client.search_messages(query=query, sort="timestamp", sort_dir="desc", count=100)




In [80]:
for x in result["messages"]["matches"]:
    print(x)

{'iid': 'f583dd1e-5af6-4bc7-bc85-94f5c1f830bb', 'team': 'T6963A864', 'score': 0, 'channel': {'id': 'C6A100T5F', 'is_channel': True, 'is_group': False, 'is_im': False, 'is_mpim': False, 'is_shared': False, 'is_org_shared': False, 'is_ext_shared': False, 'is_private': False, 'name': 'code', 'pending_shared': [], 'is_pending_ext_shared': False}, 'type': 'message', 'user': None, 'username': 'github', 'ts': '1724248000.440549', 'attachments': [{'id': 1, 'color': '6CC644', 'fallback': '[OpenMined/PySyft] Pull request submitted: "#9189 Use torch 2.2.2" https://github.com/OpenMined/PySyft/pull/9189 by kiendang', 'text': 'Torch 2.3.1 breaks syft installation for Mac with intel chips, as reported by @koenvanderveen.', 'pretext': '[OpenMined/PySyft] Pull request submitted by <https://github.com/kiendang|kiendang>', 'title': '#9189 Use torch 2.2.2', 'title_link': 'https://github.com/OpenMined/PySyft/pull/9189', 'mrkdwn_in': ['pretext', 'text']}], 'text': '', 'permalink': 'https://openmined.slack.c

In [ ]:
messages = result["messages"]["matches"]
for msg in messages:
    user = msg.get("user")
    text = msg.get("text")
    ts = msg.get("ts")
    channel = msg.get("channel", {}).get("name")
    print(f"[{ts}] in #{channel} by {user}: {text}")

In [ ]:
# from pprint import pprint

# # response = client.conversations_list(types="im")

# channels = [channel for channel in response["channels"] if channel["is_im"]]

# for channel in channels[:2]:
#     conversation_history = client.conversations_history(
#         channel=channel["id"],
#         limit=3,
#     )
#     print("USER", id2user[channel["id"]])
#     pprint(conversation_history["messages"])

In [ ]:
def get_all_messages(client, since_timestamp):
    response = client.conversations_history(
        channel="C06650000000000000000000",
        oldest=since_timestamp,
        limit=100,
    )
    return response["messages"]


import time
last_week_timestamp = int(time.time()) - 60 * 60 * 24 * 7
client = WebClient(token=token, headers=headers)
get_all_messages(client, last_week_timestamp)

In [ ]:
from pprint import pprint

res = []
for message in response["messages"]:
    res.append({
        "text": message["text"],
        "user": "Madhava (we're hiring)",
        "ts": message["ts"],
    })

pprint(res)

In [ ]:
response["channels"][:3]